In [ ]:
!pip install fake_headers
!pip install scrapy
!pip install crochet

In [ ]:
from fake_headers import Headers
import requests
import hashlib
import re
import pandas as pd
import scrapy
from crochet import setup, wait_for
from scrapy.crawler import CrawlerRunner
import argparse
import io
import imagehash
from PIL import Image

In [ ]:

classes = pd.read_csv(r"/kaggle/input/parsing-ebay/classes.csv")
model_request = []
model_name = []
OLD_ID = 262 # WHAT THAT, SERGEY??

for index, row in classes.iterrows():
    if index <= OLD_ID:
        continue
    if classes.at[index, "brand"] in classes.at[index, "model"]:
        result = classes.at[index, "model"]
    else:
        result = classes.at[index, "brand"] + " " + classes.at[index, "model"]
    url = f'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2334524.m570.l1313&_nkw={"+".join(result.split())}&_in_kw=3&_sacat=11450&_sop=12&_ipg=60&_fcid=3'
    model_request.append(url)
    model_name.append(result)
    

    
print(len(model_request))
print(model_request[0])

In [1]:
setup()
seen_sites = set()


main_df = pd.read_csv(r'hash\allhash.csv') # Dataframe with test data with column avg hash
main_df['class_id'] = -1
class_df = pd.read_csv(r'data\classes.csv') # Dataframe with column class_id and model
    

class EbaySpider(scrapy.Spider):
    """
    Our spider for parse image from ebay and analyze avg_hash them, therefore
    fill dataframe finded class_id.
    """
    
    # Naming our spider for fun :D
    name = "Serega"

    
    # 1 stage: iterate on search url
    def start_requests(self):
        meta = zip(model_request[:2], model_name[:2])
        for url, model in meta:
            yield scrapy.Request(url=url, 
                                 callback=self.parse, 
                                 cb_kwargs=dict(find_model=model))

    # 2 stage: find page url boots and iterate
    def parse(self, response, find_model):
        LIMIT_ITER = 15 # num of urls we iterate in loops

        links = response.xpath(r'//div[@class = "srp-river-results clearfix"]//@href').getall()
        for quote in links[:LIMIT_ITER]: 
            yield scrapy.Request(url=quote, 
                                 callback=self.parse_hash,
                                 cb_kwargs=dict(find_model=find_model))
    
    # 3 stage: iterate on image in page boots and analyze them average hash
    def parse_hash(self, response, find_model):
        global class_df
        global main_df
        
        tmp_model = response.xpath(r'//div[@class="ux-layout-section ux-layout-section--features"]//span[@itemprop="model"]//span[@class="ux-textspans"]/text()').get()
        tmp_model = tmp_model.lower()
        if tmp_model != find_model:
            if len(class_df[class_df.model == tmp_model]) > 0:
                print("cooon", find_model, tmp_model)

                fin = re.findall(r"https:\/\/i\.ebayimg\.com\/(?!thumbs\/)images\/g\/[a-zA-Z0-9]*\/s-l500\.jpg", str(response.text))

                session = requests.Session()
                
                for img_url in list(set(fin)):
                    
                    try:
                        img_data = session.get(img_url)
                    except:
                        headers = Headers(headers=True).generate()
                        session.headers = headers
                        img_data = session.get(img_url)


                    img_data = img_data.content
                    with Image.open(io.BytesIO(img_data)) as img:
                        avg_hash = str(imagehash.average_hash(img, 32))
                    
                    main_df.loc[(main_df['avg_hash'] == avg_hash), 'class_id'] = class_df[class_df.model == tmp_model].id.values()[0]
                
            else:
                pass

        else:
            fin = re.findall(r"https:\/\/i\.ebayimg\.com\/(?!thumbs\/)images\/g\/[a-zA-Z0-9]*\/s-l500\.jpg", str(response.text))

            session = requests.Session()
            
            for img_url in list(set(fin)):
                
                try:
                    img_data = session.get(img_url)
                except:
                    headers = Headers(headers=True).generate()
                    session.headers = headers
                    img_data = session.get(img_url)


                img_data = img_data.content
                with Image.open(io.BytesIO(img_data)) as img:
                    avg_hash = str(imagehash.average_hash(img, 32))
                
                main_df.loc[(main_df['avg_hash'] == avg_hash), 'class_id'] = class_df[class_df.model == tmp_model].id.values()[0]

        
@wait_for(10)

def run_spider():
    """run spider"""
    crawler = CrawlerRunner()
    d = crawler.crawl(EbaySpider)
    return d

run_spider()

NameError: name 'setup' is not defined